In [1]:
import jax
import jax.numpy as jnp



In [38]:
rng = jax.random.PRNGKey(0)

x = jax.random.normal(rng, (501, 49, 47))

ii = jnp.argpartition(x.max(axis=1), 5, axis=-1)[:, :5]

In [41]:
@jax.jit
def normal_take(x, i):
    return jnp.take_along_axis(x, ii[:, None, :], axis=-1)

normal_take(x, ii).shape

(501, 49, 5)

In [43]:
%%timeit

normal_take(x, ii).block_until_ready()

47.3 µs ± 303 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [35]:
import einops as ein

@jax.jit
def oh_ind(x, i):
    I = jnp.eye(x.shape[2], dtype=jnp.bool_)
    ii = I[i]
    return ein.einsum(x, ii, 'a b c, a k c -> a b k')

oh_ind(x, ii).shape

(51, 49, 5)

In [37]:
%%timeit

oh_ind(x, ii).block_until_ready()

42 µs ± 619 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
